<h1>Preparation of the German Twitter Data</h1>

<p>This file is for preparing the german tweets. The preparation is mainly for training my own model for sentiment detection with Flair. </p>

In [2]:
import csv
import numpy as np
import pandas as pd
import re
import nltk

<h1>Einlesen und bestimmung der Felder</h1>
<p>Durch die weitern Operatoren, kann ich die einzelnen Felder in DF bestimmen. Durch die Lambda Funktion werden die Str zu Listen, was die spätere verarbeitung erleichtert. 
    Durch die Parse_dates Funktion wird das Datumsfeld zu einem Timestamp, was die weiter Verarbeitung wesentlich leichter macht.</p>

In [256]:
df_manager_de = pd.read_csv("/Users/tobias/Dev/FOM/Master_Thesis/data/Manager_DE/data.csv", sep=";", 
                            converters={"tweet_type": lambda x: x.strip("[]").replace("'", "").split(", "),
                                        "referenced_tweet_id": lambda x: x.strip("[]").replace("'", "").split(", ")},
                           parse_dates=["created_at"])
df_manager_de = df_manager_de.explode(["referenced_tweet_id", "tweet_type"], ignore_index=True)

In [257]:
df_manager_de.head()

,conversation_id,author_id,created_at,tweet_id,lang,retweet_count,reply_count,like_count,quote_count,source,tweet_type,referenced_tweet_id,text
0,703311366262800384,1392131804,2016-02-26 20:11:11+00:00,703311366262800384,de,0,0,2,0,Twitter Web Client,None,None,#Wirecard-Chef Markus Braun pumpt irrwitzige S...
1,702405572012974080,1053723204,2016-02-24 08:11:53+00:00,702405572012974080,de,0,0,0,0,Twitter Web Client,None,None,Jetzt günstig einsammeln und später absahnen? ...
2,701784983829143553,554491983,2016-02-22 15:05:53+00:00,701784983829143553,de,0,0,0,0,Facebook,None,None,Morgen startet die neue Reihe R²-JazzCube! Vol...
3,701446667170615296,3365669884,2016-02-21 16:41:32+00:00,701446667170615296,de,0,0,1,0,Twitter Web Client,None,None,die Y-Achse des #prusai3 nimmt Formen an :-) ...
4,700597812556386304,4858383167,2016-02-19 08:28:29+00:00,700597812556386304,de,2,0,0,0,Twitter for iPhone,retweeted,700578193334411264,RT @aktionaer: #Wirecard: Was führt der Vorsta...


<br>
<p>Prüfen welche Datenfelder in den einzelnen Zeilen enthalten sind. War wichtig um diese anpassen zu können.</p>

In [204]:
print("conversation_id: ", df_manager_de.iloc[0]["conversation_id"], "type: ", type(df_manager_de.iloc[0]["conversation_id"]))
print("author_id: ", df_manager_de.iloc[0]["author_id"], "type: ", type(df_manager_de.iloc[0]["author_id"]))
print("created_at: ", df_manager_de.iloc[0]["created_at"], "type: ", type(df_manager_de.iloc[0]["created_at"]))
print("tweet_id: ", df_manager_de.iloc[0]["tweet_id"], "type: ", type(df_manager_de.iloc[0]["tweet_id"]))
print("lang: ", df_manager_de.iloc[0]["lang"], "type: ", type(df_manager_de.iloc[0]["lang"]))
print("retweet_count: ", df_manager_de.iloc[0]["retweet_count"], "type: ", type(df_manager_de.iloc[0]["retweet_count"]))
print("reply_count: ", df_manager_de.iloc[0]["reply_count"], "type: ", type(df_manager_de.iloc[0]["reply_count"]))
print("like_count: ", df_manager_de.iloc[0]["like_count"], "type: ", type(df_manager_de.iloc[0]["like_count"]))
print("quote_count: ", df_manager_de.iloc[0]["quote_count"], "type: ", type(df_manager_de.iloc[0]["quote_count"]))
print("source: ", df_manager_de.iloc[0]["source"], "type: ", type(df_manager_de.iloc[0]["source"]))
print("tweet_type: ", df_manager_de.iloc[0]["tweet_type"], "type: ", type(df_manager_de.iloc[0]["tweet_type"]))
print("referenced_tweet_id: ", df_manager_de.iloc[0]["referenced_tweet_id"], "type: ", type(df_manager_de.iloc[0]["referenced_tweet_id"]))
print("text: ", df_manager_de.iloc[0]["text"], "type: ", type(df_manager_de.iloc[0]["text"]))

conversation_id:  703311366262800384 type:  <class 'numpy.int64'>
author_id:  1392131804 type:  <class 'numpy.int64'>
created_at:  2016-02-26 20:11:11+00:00 type:  <class 'pandas._libs.tslibs.timestamps.Timestamp'>
tweet_id:  703311366262800384 type:  <class 'numpy.int64'>
lang:  de type:  <class 'str'>
retweet_count:  0 type:  <class 'numpy.int64'>
reply_count:  0 type:  <class 'numpy.int64'>
like_count:  2 type:  <class 'numpy.int64'>
quote_count:  0 type:  <class 'numpy.int64'>
source:  Twitter Web Client type:  <class 'str'>
tweet_type:  None type:  <class 'str'>
referenced_tweet_id:  None type:  <class 'str'>
text:  #Wirecard-Chef Markus Braun pumpt irrwitzige Summen in sein Unternehmen. Woher stammt das Geld? @SchreiberDohms  https://t.co/VdsV4nvBe4 type:  <class 'str'>


<h3>Löschen von unnötigen Informationen</h3>
<p>Hier werden Informationen aus den Tweets Texten, die nicht weiter benötigt werden. Bspw. Links</p>

In [258]:
for each in range(len(df_manager_de)):
    df_manager_de.loc[df_manager_de["text"] == 
                      df_manager_de.loc[each]["text"], "text"] = re.sub(r'http\S+', '', 
                                                                        (re.sub('@[^\s]+', '',
                                                                                re.sub('#', '', 
                                                                                       df_manager_de.loc[each]["text"]))))

In [259]:
df_manager_de.iloc[0]["text"]

'Wirecard-Chef Markus Braun pumpt irrwitzige Summen in sein Unternehmen. Woher stammt das Geld?   '

<br>
<h3>Nicht Kontextbezogene Tweets löschen</h3>
<p>So kann ich prüfen ob bestimmte Begriffe in den Texten enthalten sind und mir die conversation_id ausgeben lassen. Diese lasse ich dann löschen und auch zugehörige Tweets, da diese nicht in den Kontext passen.<br>Die wörter in der Liste müssen klein geschrieben werden, da auch alle Buchstaben für die Prüfung klein geschrieben werden müssen, da sinst nicht alles erkannt wird.</p>
<p>Zu löschende Begriffe</p>
<ul>
    <li>BVB</li>
    <li>Fußball</li>
    <li>Fussball</li>
    <li>Dortmund</li>
</ul>

In [260]:
not_used_words_list = ["bvb", "fußball", "fussball", "dortmund"]
tweet_id_without_context_list = []
for each in range(len(df_manager_de)):
    for word in not_used_words_list:
        if word in df_manager_de.iloc[each]["text"].lower():
            if df_manager_de.iloc[each]["tweet_id"] not in tweet_id_without_context_list:
                tweet_id_without_context_list.append(df_manager_de.iloc[each]["tweet_id"])

<p>Iteration durch die Liste mit zu löschenden Tweets, bis der Wert zu Beginn und zum Ende gleich ist und es keine
weiteren Tweets mehr gibt, die gefiltert werden müssen. </p>

In [261]:
def check_for_further_tweets():
    start_len = len(tweet_id_without_context_list)
    for each in range(len(tweet_id_without_context_list)):
        df_tweets = df_manager_de.loc[(df_manager_de["conversation_id"] == tweet_id_without_context_list[each]) | 
               (df_manager_de["referenced_tweet_id"] == str(tweet_id_without_context_list[each]))]

        for each in range(len(df_tweets)):
            if df_tweets.iloc[each]["tweet_id"] not in tweet_id_without_context_list:
                tweet_id_without_context_list.append(df_tweets.iloc[each]["tweet_id"])
                
    if start_len != len(tweet_id_without_context_list):
        check_for_further_tweets()
check_for_further_tweets()

<p>Löschen der nicht kontextbezogenen Tweets und den Index neu setzen um später keine Probleme mit dem Index zu bekommen.</p>

In [262]:
for each in tweet_id_without_context_list:
    df_manager_de = df_manager_de.drop(df_manager_de.loc[df_manager_de["tweet_id"] == each].index[0])
df_manager_de = df_manager_de.reset_index(drop=True)

,conversation_id,author_id,created_at,tweet_id,lang,retweet_count,reply_count,like_count,quote_count,source,tweet_type,referenced_tweet_id,text
0,703311366262800384,1392131804,2016-02-26 20:11:11+00:00,703311366262800384,de,0,0,2,0,Twitter Web Client,None,None,Wirecard-Chef Markus Braun pumpt irrwitzige Su...
1,702405572012974080,1053723204,2016-02-24 08:11:53+00:00,702405572012974080,de,0,0,0,0,Twitter Web Client,None,None,Jetzt günstig einsammeln und später absahnen? ...
2,701784983829143553,554491983,2016-02-22 15:05:53+00:00,701784983829143553,de,0,0,0,0,Facebook,None,None,Morgen startet die neue Reihe R²-JazzCube! Vol...
3,701446667170615296,3365669884,2016-02-21 16:41:32+00:00,701446667170615296,de,0,0,1,0,Twitter Web Client,None,None,die Y-Achse des prusai3 nimmt Formen an :-) ...
4,700597812556386304,4858383167,2016-02-19 08:28:29+00:00,700597812556386304,de,2,0,0,0,Twitter for iPhone,retweeted,700578193334411264,RT Wirecard: Was führt der Vorstand im Schild...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14402,1278189179327954944,212758794,2020-07-01 04:50:22+00:00,1278189179327954944,de,0,0,0,0,Twitter Web App,None,None,Wirecard wird womöglich gestückelt an Investor...
14403,1278182059572432896,58417079,2020-07-01 04:22:05+00:00,1278182059572432896,de,0,0,1,0,LinkedIn,None,None,Das Ende eines großen Traums zeichnet sich ab....
14404,1278164531613782017,386165847,2020-07-01 03:12:26+00:00,1278164531613782017,de,0,0,0,0,IFTTT,None,None,"Wirecard kündigt Markus Braun, Insolvenzverwal..."
14405,1278158825661566979,1245423304304926727,2020-07-01 02:49:45+00:00,1278158825661566979,de,0,0,0,0,Blog2Social APP,None,None,Kündigung für Markus Braun: Der Aufsichtsrat v...


<br>
<h4>Check</h4>
<p>Aufrufen einer Line um zu prüfen, welche Informationen enthalten sind.<br>
    Den Index abrufen um mir nur den Text in voller Länge anzeigen zu lassen, da dieser sonst gekürzt ist.</p>
   

In [69]:
df_manager_de.loc[df_manager_de["conversation_id"] == 1098841597582737409]

,conversation_id,author_id,created_at,tweet_id,lang,retweet_count,reply_count,like_count,quote_count,source,tweet_type,referenced_tweet_id,text


In [49]:
df_manager_de.loc[df_manager_de.index[df_manager_de["conversation_id"] == 1278435775441494016].tolist()[0]]["text"]

'RT @RNBVB: 👨🏻\u200d⚕️ Dr. Markus #Braun ist vom Nachrichtenmagazin #Focus als "Top-Mediziner" in Deutschland ausgezeichnet worden. Zuletzt arbei…'

In [48]:
print(df_manager_de.index[df_manager_de["conversation_id"] == 1278435775441494016].tolist()[0])

14209


In [36]:
df_manager_de.loc[14209]["text"]

'RT @RNBVB: 👨🏻\u200d⚕️ Dr. Markus #Braun ist vom Nachrichtenmagazin #Focus als "Top-Mediziner" in Deutschland ausgezeichnet worden. Zuletzt arbei…'

<br>
<h3>Löschen von Retweets</h3>
<p>Alle Retweets löschen, da diese in diesem Schritt nicht benötigt werden.</p>

In [266]:
counter = 0
retweet_list = []
for each in range(len(df_manager_de)):
    if "retweeted" in df_manager_de.iloc[each]["tweet_type"]:
        retweet_list.append(each)
        counter += 1
print(counter)

5428


In [278]:
df_manager_de = df_manager_de.drop(retweet_list)

<h1>Check User</h1>
<p>Überprüfe die User Daten nach Namen, die nichts mit dem Thema Wirecard zu tun haben. Auch Nutzer, die ggf. nur durch Zufall in die Abfrage gekommen sind, z.B. durch den gleichen Namen wie einer der Manager, sollen aus dem Datensatz entnommen werden.</p>

In [7]:
df_user = df_manager_de = pd.read_csv("/Users/tobias/Dev/FOM/Master_Thesis/data/Manager_DE/user.csv", sep=";")

In [279]:
df_user["name"].value_counts().head(30)

Markus Braun                                   44
Handelsblatt                                   29
Wirecard                                       27
kein Spekulant blauer Haken                    26
FAZ Finanzen                                   26
Dr. Fred... no MD/PD/XYZ-D                     25
Ehssan Khazaeli                               25
AMCtoooMoon                                    25
HandelsblattFinanzen                           24
manager magazin                                24
csfa.                                          22
A.Barendrecht                                  21
Daniel                                         20
cS-Strategies                                  20
Frankfurter Allgemeine gesamt                  20
Mario                                          20
Nils Wischmeyer                                20
FinanzNachrichten.de                           20
🎲 Policon automatisches Nachrichtenportal 👽    19
WirtschaftsWoche                               19


<p><br><b>Check nach besonderen Namen, die in der Liste besonder oft auftauchen oder Namen, denen eine besondere Bedeutung zugeordnet wird.</b></p>

In [174]:
df_user.loc[df_user.name.str.contains("markus")].drop_duplicates()

,user_id,username,name,follower,following,tweet_cound,list_count,verified,created_at
4888,443638494,markuskrainer1,markus krainer,2,34,10,0,False,2011-12-22 11:25:26+00:00
4953,46415147,kubant,markus,64,340,2786,0,False,2009-06-11 15:46:37+00:00


<p><b>Tweets von den zu prüfenden Leuten ansehen</b></p>

In [194]:
user_id_list = []
for each in df_user.loc[df_user.name.str.lower().str.contains("jan")].drop_duplicates()["user_id"]:
    user_id_list.append(each)
df_manager_de.loc[df_manager_de.author_id.isin(user_id_list)]

,conversation_id,author_id,created_at,tweet_id,lang,retweet_count,reply_count,like_count,quote_count,source,tweet_type,referenced_tweet_id,text
196,1039392642847535104,152703288,2018-09-11 05:58:11+00:00,1039392642847535104,de,0,0,1,0,LinkedIn,None,None,Speaker Coup: Knapp nach DAX-Listing: Wirecard...
197,1039176827049332736,152703288,2018-09-10 15:40:37+00:00,1039176827049332736,de,2,0,0,0,Twitter for iPhone,retweeted,1039065678353973248,RT @DarwinsCircle: Knapp nach DAX-Listing: Wir...
602,1110837535423504384,960630599601082368,2019-03-27 10:11:37+00:00,1110846829795729408,de,0,1,1,0,Twitter for iPhone,replied_to,1110840122201784320,@jens_algebra @_MarkusBraun Das ist doch jetzt...
1039,1116041090677784578,571011183,2019-04-10 20:15:23+00:00,1116072202288467968,de,0,1,1,0,Twitter Web App,replied_to,1116065141014052864,@AuchDuMeinSohn @Alexand93433205 @Zwitscherpau...
1058,1114423943761473536,571011183,2019-04-06 10:05:16+00:00,1114469109914374144,de,0,0,0,0,Twitter Web App,replied_to,1114467448647028736,@AuchDuMeinSohn @bald_saitama @aveirodobraga @...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14034,1279469432948088832,1261872733,2020-07-04 17:37:38+00:00,1279469432948088832,de,1,0,0,0,Hootsuite Inc.,retweeted,1279467577413193728,RT @welt: Gefälschte Einreiseunterlagen auf de...
14279,1278761776377663489,804934696555704320,2020-07-02 18:45:40+00:00,1278761776377663489,de,19,0,0,0,Twitter for iPhone,retweeted,1278757395892703235,RT @SZ: Die weltweite Suche nach Ex-Wirecard-V...
14291,1278749709469913091,972233251,2020-07-02 17:57:43+00:00,1278749709469913091,de,1,0,0,0,Twitter Web App,retweeted,1278748251315937280,RT @VinzentLeitgeb: Eine Donald-Trump-Pappfigu...
14305,1278732404497223681,48455041,2020-07-02 16:48:57+00:00,1278732404497223681,de,3,0,0,0,Twitter for Android,retweeted,1278729605696884738,"RT @wischmeyer_n: Jan Marsalek, Ex-Vorstand vo..."
